# TAT-C: Collect Observations Core Functions Example

Dr. Paul Grogan, I. Josue Tapia-Tamayo, Isaac Feldman
Collective Design (CoDe) Lab
Stevens Institute of Technology, School of Systems and Enterprises


This example demonstrates how to use direct function calls of the low-level TAT-C library to model observations of single point  by a single satellite using an orbit derived from an existing Two Line Element (TLE) set.

## Dependencies

### Standard Python Libraries
This example is compatible with python 3.8.10 and makes use of the standard python `datetime` library for setting mission duration.

In [8]:
# Import datetime library objects for seting mission duration
from datetime import datetime, timezone, timedelta

### TAT-C Library
Additionally, this example makes use of the low-level `tatc` library. If you do not have this installed, navigate to the main `tatc-v3` file and make sure you have followed all the instructions for installation of TAT-C.

In [9]:
# Import tatc core functions and schemas
from tatc.analysis.coverage import collect_observations, reduce_observations
from tatc.schemas.satellite import Satellite
from tatc.schemas.instrument import Instrument
from tatc.schemas.orbit import TwoLineElements
from tatc.schemas.point import Point

## Establish the Mission Architecture and Key Information
The first step in modeling a mission with TAT-C is to establish the key mission parameters -- specifically the satellites, the mission time frame and points on the ground for observation.

### Modeling Satellite Architecture
First we define the satellites for a mission. In this example we define one satellite (NOAA-1) with a single instrument and an orbit derived from a known two-line element set (TLE). This uses the `TwoLineElements` class from `tatc.schemas.orbit` and the `Instrument` class from `tatc.schemas.instrument`.

In [10]:
# Save the NOAA-1 TLE as a list where the first element of the list is the first line
# of the TLE and the second list element is the second line
tle = ['1 04793U 70106A   22044.41526573 -.00000035  00000+0  54783-4 0  9991',
        '2 04793 101.5750 111.2777 0031771 203.0437 167.8944 12.54003052342042']

# Derive the satellite orbit from the TLE
orbit = TwoLineElements(type='tle',
                        tle=tle)

# Initialize the instrument for the satellite to make observations
instrument = Instrument(name='Lead Instrument', field_of_regard=180)

# Define the satellite from the orbit and instrument
sat = Satellite(name='NOAA1',
                type='satellite',
                orbit=orbit,
                instruments=[instrument])

### Modeling Mission Parameters
Next we define the mission parameters, specifically the target point -- using the `Point` class from `tatc.schemas.point` -- and the start and end time of the mission window -- using the `datetime` and `timedelta` objects from the python `datetime` module.

In [11]:
# Define a target points for observations
point = Point(id=0, latitude=40.74259, longitude=-74.02686)

# Set the start date to be January, 1, 2021 at noon (12 pm) UTC
start = datetime(year=2021, month=1, day=1, hour=12, minute=0, second=0, tzinfo=timezone.utc)

# Set the end date to be 30 days after the start date
end = start + timedelta(days=30)

## Calling Analysis Functions
Finally, we call the core analysis function `collect_observations` function from `tatc.analysis.coverage` to determine all the opportunities for the satellite to observe the point with the specified instrument over the mission duration.

### Collect Observations

In [12]:
# Collect and display the observation events for the point over the mission duration
results = collect_observations(point, sat, instrument, start, end, omit_solar=False)
results

,point_id,geometry,satellite,instrument,start,end,epoch,sat_alt,sat_az,sat_sunlit,solar_alt,solar_az,solar_time,access,revisit
0,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-01 12:42:50.676073+00:00,2021-01-01 12:51:54.832526+00:00,2021-01-01 12:47:24.047915+00:00,2.586295,79.509428,True,3.501823,124.642954,7.793303,0 days 00:09:04.156453,NaT
1,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-01 18:18:10.127555+00:00,2021-01-01 18:33:28.661941+00:00,2021-01-01 18:25:49.255502+00:00,10.754665,318.864281,True,23.262669,201.534585,13.431978,0 days 00:15:18.534386,0 days 05:50:37.985868
2,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-01 23:41:57.998882+00:00,2021-01-01 23:57:59.674415+00:00,2021-01-01 23:49:58.234982+00:00,12.349392,43.058003,False,-23.895856,259.604899,18.832813,0 days 00:16:01.675533,0 days 05:39:49.546860
3,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-02 05:26:29.034096+00:00,2021-01-02 05:31:29.515226+00:00,2021-01-02 05:28:58.374827+00:00,0.735631,282.345132,False,-71.145244,20.973091,0.480882,0 days 00:05:00.481130,0 days 05:49:31.516344
4,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-02 13:32:12.735122+00:00,2021-01-02 13:50:09.066641+00:00,2021-01-02 13:41:15.786290+00:00,15.328477,88.982908,True,11.388286,134.155423,8.682986,0 days 00:17:56.331519,0 days 08:23:40.032545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-30 17:21:01.667535+00:00,2021-01-30 17:39:59.783792+00:00,2021-01-30 17:30:32.811973+00:00,22.866600,307.711367,True,31.594270,185.908479,12.351587,0 days 00:18:58.116257,0 days 04:04:08.419761
164,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-30 19:15:29.287014+00:00,2021-01-30 19:24:30.876714+00:00,2021-01-30 19:20:00.073797+00:00,3.037353,332.159462,True,24.403038,214.401438,14.175695,0 days 00:09:01.589700,0 days 02:03:29.209179
165,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-30 22:49:51.453057+00:00,2021-01-30 23:00:30.980105+00:00,2021-01-30 22:55:11.807304+00:00,4.357143,30.882562,True,-8.631651,254.436052,17.761980,0 days 00:10:39.527048,0 days 03:45:01.693091
166,0,POINT (-74.02686 40.74259),NOAA1,Lead Instrument,2021-01-31 00:34:58.355158+00:00,2021-01-31 00:54:47.997871+00:00,2021-01-31 00:44:53.818314+00:00,26.731959,54.604082,False,-29.161200,271.683954,19.590121,0 days 00:19:49.642713,0 days 02:04:56.544814


### Reducing Observations

We can also utilize the `reduce_observations` function from `tatc.analysis.coverage` to generate coverage statistics -- total access time, average revisit time, and the number of observations for the point over the mission duration.

In [13]:
# Calculate and displace coverage statistics for the point over the mission duration
reduced_results = reduce_observations(results)
reduced_results

,point_id,geometry,access,revisit,num_obs
point_id,,,,,
0,0,POINT (-74.02686 40.74259),0 days 00:14:50.879800,4.509454,168
